In [1]:
# import packages 
import os, sys, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from scipy.stats import pearsonr

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from DGXutils import GetLowestGPU

sys.path.append('../')

from utils.dataset import *
from utils.model import *

In [2]:
# load data
gxe_test = GxE_Dataset(split='test')
test_loader = DataLoader(gxe_test,
                         batch_size=64,
                         num_workers=64,
                         shuffle=True)

In [31]:
# load model + weights
device = GetLowestGPU()

checkpoint_path = '../checkpoints/gxe_model/checkpoint_6111.pt'
checkpoint = torch.load(checkpoint_path)["model"]

model = GxE_Transformer(config=TransformerConfig).to(device)
model.load_state_dict(checkpoint)

/tmp/ipykernel_533927/4062979175.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)["model"]


<All keys matched successfully>

In [32]:
# evaluate on test set
preds = []
actuals = []

# loop through
for xb, yb in tqdm(test_loader):
    
    # get things on device
    for key, value in xb.items():
        xb[key] = value.to(device)

    preds.extend(model(xb).detach().tolist())
    actuals.extend(yb.tolist())

  0%|          | 0/437 [00:02<?, ?it/s]

In [37]:
pearsonr(preds, actuals)

PearsonRResult(statistic=array([0.76931704]), pvalue=array([0.]))